In [7]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from decouple import config

In [10]:
from casestudy.update import auto

In [11]:
from casestudy.update.baseframe import make

In [12]:
auto()

Instantiate new exception logger
Running italy
Running braz


KeyboardInterrupt: 